In [46]:
from datetime import datetime
from calendar import monthrange
import pytz
import re

# Functions

In [92]:
def is_NERC(date):
    ## Memorial Day
    if date.month == 5 and date.weekday() == 0 and 25 <= date.day <= 31:
        return True
    
    ## Labor Day
    if date.month == 9 and date.weekday() == 0 and 1 <= date.day <= 7:
        return True
    
    ##Thanksgiving Day
    if date.month == 11 and date.weekday() == 3 and 22 <= date.day <= 28:
        return True
    
    ## New Year's Day
    if date.month == 1 and date.day == 1 and date.weekday() != 6:
        return True
    if date.month == 1 and date.day == 2 and date.weekday() == 0:
        return True
    
    ## Independence Day's Day
    if date.month == 7 and date.day == 4 and date.weekday() != 6:
        return True
    if date.month == 7 and date.day == 5 and date.weekday() == 0:
        return True
    
    ## Christmas Day
    if date.month == 12 and date.day == 25 and date.weekday() != 6:
        return True
    if date.month == 12 and date.day == 26 and date.weekday() == 0:
        return True
    
    return False
    

In [48]:
def get_daily(iso, peak_type, period,east_indicator,temp):
    
    period = datetime.strptime(period, '%Y-%m-%d')

    if peak_type == '7x8':
        ## starting date of day light saving
        if iso != 'MISO' and period.month in [2,3,4] and period in temp: 
            res = 7
        ## ending date of day light saving
        elif iso != 'MISO' and period.month in [9,10,11] and period in temp:
            res = 9
        else:
            res = 8
    
    elif peak_type == 'flat':
        res = 24
    
    else:
        ## Two situations, Western market takes Saturday as a weekday and Eastern market not
        if east_indicator == 1:
            if period.weekday()+1 <= 5 and not is_NERC(period):
                if peak_type == 'onpeak':
                    res = 16

                elif peak_type == 'offpeak':
                    if iso != 'MISO' and period.month in [2,3,4] and period in temp: 
                        res = 7
                    ## ending date of day light saving
                    elif iso != 'MISO' and period.month in [9,10,11] and period in temp:
                        res = 9
                    else:
                        res = 8


                elif peak_type == '2x16H':
                    res = 0

            else:
                if peak_type == 'onpeak':
                    res = 0

                if peak_type == 'offpeak':
                    if iso != 'MISO' and period.month in [2,3,4] and period in temp: 
                        res = 23
                    ## ending date of day light saving
                    elif iso != 'MISO' and period.month in [9,10,11] and period in temp:
                        res = 25
                    else:
                        res = 24

                if peak_type == '2x16H':
                    res = 16

        else:
            if period.weekday()+1 <= 6 and not is_NERC(period):
                if peak_type == 'onpeak':
                    res = 16

                elif peak_type == 'offpeak':
                    if iso != 'MISO' and period.month in [2,3,4] and period in temp: 
                        res = 7
                    ## ending date of day light saving
                    elif iso != 'MISO' and period.month in [9,10,11] and period in temp:
                        res = 9
                    else:
                        res = 8


                elif peak_type == '2x16H':
                    res = 0

            else:
                if peak_type == 'onpeak':
                    res = 0

                if peak_type == 'offpeak':
                    if iso != 'MISO' and period.month in [2,3,4] and period in temp: 
                        res = 23
                    ## ending date of day light saving
                    elif iso != 'MISO' and period.month in [9,10,11] and period in temp:
                        res = 25
                    else:
                        res = 24

                if peak_type == '2x16H':
                    res = 16
    
    return res
        

In [49]:
def get_monthly(iso, peak_type, period,east_indicator,temp):
    pattern = r'([0-9]{4})(\w{3})'
    match = re.match(pattern, period)
    year = match.groups()[0]
    month = match.groups()[1]
    month = datetime.strptime(month,'%b').month
    res = 0
    for i in range(1,monthrange(int(year), month)[1]+1):
        period = year + '-' + str(month) + '-' + str(i)
        res += get_daily(iso, peak_type, period,east_indicator,temp)
    start_date = year + '-' + str(month) + '-' + str(1)
    end_date = period
    return [start_date,end_date,res]
    

In [50]:
def get_quarterly(iso, peak_type, period,east_indicator,temp):
    pattern = r'([0-9]{4})Q([1-4]{1})'
    match = re.match(pattern, period)
    year = match.groups()[0]
    month = int(match.groups()[1])
    month = [i for i in range(3*month-2,3*month+1)]
    res = 0
    for month_ in month:
        for i in range(1,monthrange(int(year), month_)[1]+1):
            period = year + '-' + str(month_) + '-' + str(i)
            res += get_daily(iso, peak_type, period,east_indicator,temp)
    start_date = year + '-' + str(month[0]) + '-' + str(1)
    end_date = period
    return [start_date,end_date,res]   

In [70]:
def get_yearly(iso, peak_type,period,east_indicator,temp):
    year = int(period[:4])
    res = 0
    for month_ in range(1,13):
        for i in range(1,monthrange(year, month_)[1]+1):
            period = str(year) + '-' + str(month_) + '-' + str(i)
            res += get_daily(iso, peak_type, period,east_indicator,temp)
    start_date = str(year) + '-' + str(1) + '-' + str(1)
    end_date = period
    return [start_date,end_date,res]   


In [71]:
def get_hours(iso, peak_type, period):
    ## List of starting/ending dates of day light saving time
    localtime = pytz.timezone('US/Central') ## The timezone could be changed according to iso 
    list_ = localtime._utc_transition_times
    for i in range(len(list_)):
        list_[i] = list_[i].replace(hour=0)
        
    ## Decide area
    indicator = 0
    if iso in ['PJMISO', 'MISO', 'ERCOT', 'SPPISO', 'NYISO']:
        indicator = 1
        
    if period[4] == '-':
        
        res = get_daily(iso, peak_type, period,indicator,list_)
        return [iso,peak_type,period,period,res]
    
    if len(period) == 7:
        res = get_monthly(iso, peak_type, period,indicator,list_)
        temp = [iso,peak_type]
        temp.extend(res)
        return temp
    
    elif period[4] == 'Q':
        res = get_quarterly(iso, peak_type, period,indicator,list_)
        temp = [iso,peak_type]
        temp.extend(res)
        return temp
    
    elif period[4] == 'A':
        res = get_yearly(iso, peak_type, period,indicator,list_)
        temp = [iso,peak_type]
        temp.extend(res)
        return temp


# Test

In [72]:
## Test day light saving
get_hours("ERCOT", "offpeak", "2020-11-1")

['ERCOT', 'offpeak', '2020-11-1', '2020-11-1', 25]

In [73]:
## Test holiday
get_hours("ERCOT", "offpeak", "2020-1-1")

['ERCOT', 'offpeak', '2020-1-1', '2020-1-1', 24]

In [74]:
## Test monthly sum
get_hours("ERCOT", "onpeak", "2019May")

['ERCOT', 'onpeak', '2019-5-1', '2019-5-31', 352]

In [75]:
## Test quarterly sum
get_hours("ERCOT", "offpeak", "2018Q2")

['ERCOT', 'offpeak', '2018-4-1', '2018-6-30', 1160]

In [94]:
## Test yearly sum
get_hours("ERCOT", "onpeak", "2018A")

['ERCOT', 'onpeak', '2018-1-1', '2018-12-31', 4080]